In [306]:
import requests
import pandas as pd
import json
import time

# Testing Review Shake API

In [307]:
# import list of businesses to scrape
scrape_df = pd.read_csv('Companies_to_scrape.csv')
scrape_df.head()

,Company,Category,Area,URL,Unnamed: 4,Unnamed: 5
0,Alexandra's Designer Consignment,Niche Shopping,"Seattle / Belleve, WA",https://www.yelp.com/biz/alexandras-designer-c...,NaN,NaN
1,Amore Coffee,Coffee Shop,"Seattle / Belleve, WA",https://www.yelp.com/biz/amore-coffee-bellevue...,NaN,NaN
2,Ampersand Café,Coffee Shop,"Seattle / Belleve, WA",https://www.yelp.com/biz/ampersand-cafe-seattl...,NaN,8910.0
3,Analog Coffee,Coffee Shop,"Seattle / Belleve, WA",https://www.yelp.com/biz/analog-coffee-seattle...,NaN,NaN
4,Anchored Ship Coffee Bar,Coffee Shop,"Seattle / Belleve, WA",https://www.yelp.com/biz/anchored-ship-coffee-...,NaN,NaN


## Test on Single Yelp Page

First, we weill test our function on a single coffee shop in Seattle. According to API documentation for Revew Shake, our first step is to post a job to the scraping queue. 

### Post Request for Scraping

In [3]:
# profile website
url = "https://app.datashake.com/api/v2/profiles/add"

# url of yelp page to scrape
querystring = {"url":"https://www.yelp.com/biz/armistice-coffee-roaster-seattle?osq=coffee%20shops"}

# provide api key
headers = {
    'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
    }

# request and print response
response = requests.request("POST", url, headers=headers, params=querystring)
print(response.text)

{"success":true,"job_id":118397410,"status":200,"message":"Added this profile to the queue..."}


In [211]:
response.text

'{"success":false,"status":200,"message":"We do not support app.datashake.com yet, please submit it to our feedback forum (https://feedback.reviewshake.com)"}'

In [210]:
# obtain job_id... MAKE SURE OT APPEND THIS TO LIST OF JOB ID'S SO WE MAY ITERATE OVER IT WHEN REQUESTING REVIEWS
job_id_test = json.loads(response.text)['job_id']
job_id_test

KeyError: 'job_id'

### Request Review Data

In [ ]:
# create master dataframe to add all review data to
master_df = pd.DataFrame(columns = ['business_name','source_url', 'review_ratings','review_text'])

In [121]:
url = "https://app.datashake.com/api/v2/profiles/reviews"

querystring = {"job_id":"{}".format(job_id_test),
               "page": '1'}

headers = {
    'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
    }

response_reviews = requests.request("GET", url, headers=headers, params=querystring)

print(response_reviews.text)

{"success":true,"status":200,"job_id":118397410,"source_url":"https://www.yelp.com/biz/armistice-coffee-roaster-seattle?osq=coffee shops","place_id":null,"external_identifier":null,"meta_data":"{\"address\":\"2201 Eastlake Ave E Ste A, Seattle, WA 98102\",\"name\":\"Armistice Coffee Roaster\"}","unique_id":"GD5hUn83lcOPOBl7y4p54A","review_count":53,"average_rating":4.5,"last_crawl":"2020-06-02","crawl_status":"complete","percentage_complete":100,"result_count":53,"credits_used":63,"from_date":null,"blocks":null,"reviews":[{"id":911512398,"name":"Brit P.","date":"2020-03-19","rating_value":5.0,"review_text":"They have one of the best chai lattes I've been able to find in Seattle. Very nice interior and decor. The staff is always super friendly. Love this place","url":"https://www.yelp.com/biz/armistice-coffee-roaster-seattle?hrid=bukIRIoV27crnV-AeI89cQ","profile_picture":"https://s3-media0.fl.yelpcdn.com/photo/4jxBWNH5jjfN54xMXxDLMw/180s.jpg","location":null,"review_title":null,"verifie

In [122]:
# convert response to json object
reviews_test = json.loads(response_reviews.text)
reviews_test

{'success': True,
 'status': 200,
 'job_id': 118397410,
 'source_url': 'https://www.yelp.com/biz/armistice-coffee-roaster-seattle?osq=coffee shops',
 'place_id': None,
 'external_identifier': None,
 'meta_data': '{"address":"2201 Eastlake Ave E Ste A, Seattle, WA 98102","name":"Armistice Coffee Roaster"}',
 'unique_id': 'GD5hUn83lcOPOBl7y4p54A',
 'review_count': 53,
 'average_rating': 4.5,
 'last_crawl': '2020-06-02',
 'crawl_status': 'complete',
 'percentage_complete': 100,
 'result_count': 53,
 'credits_used': 63,
 'from_date': None,
 'blocks': None,
 'reviews': [{'id': 911512398,
   'name': 'Brit P.',
   'date': '2020-03-19',
   'rating_value': 5.0,
   'review_text': "They have one of the best chai lattes I've been able to find in Seattle. Very nice interior and decor. The staff is always super friendly. Love this place",
   'url': 'https://www.yelp.com/biz/armistice-coffee-roaster-seattle?hrid=bukIRIoV27crnV-AeI89cQ',
   'profile_picture': 'https://s3-media0.fl.yelpcdn.com/photo/4j

Now let's practice obtaining the data we need. 

### Extracting Data from Review Response into Dataframe

In [123]:
# grab source url
source_url_test = reviews_test['source_url']
print(source_url_test)

# grab business name
business_name_test = json.loads(reviews_test['meta_data'])['name']
print(business_name_test)

https://www.yelp.com/biz/armistice-coffee-roaster-seattle?osq=coffee shops
Armistice Coffee Roaster


In [124]:
# review text reviews.. only twenty reviews grabbed? 
len(reviews_test['reviews'])

20

In [125]:
# grab review text data
review_text_test = []

for review in reviews_test['reviews']:
    review_text_test.append(review['review_text'])
    
review_text_test

["They have one of the best chai lattes I've been able to find in Seattle. Very nice interior and decor. The staff is always super friendly. Love this place",
 "Great experience while working. You can get work done here and get coffee or beer. First place I've seen a beer and coffee bar. Super great vibes here in Seattle.",
 "LOVE THIS PLACE. If I could live here I would- feels just like a coffee shop in Copenhagen with the little succulents and warm lighting. I don't think I've been to a more cozy coffee shop in Seattle- feels more like a home than a coffee shop. Highly recommend!",
 "Three tries at coffee and basic breakfast, three times order messed up and baristas weird with me. I'll walk literally to the other end of the block and get my coffee there. Sad that my local place has turned out to suck",
 "I love Armistice because they've nailed the combo of great coffee, bites, service, and atmosphere. \xa0Check it out",
 'I\'m currently at this cafe...right now :) Very VERY aesthetic

In [126]:
# grab review rating
review_ratings_test = []

for review in reviews_test['reviews']:
    review_ratings_test.append(review['rating_value'])
    
review_ratings_test

[5.0,
 5.0,
 5.0,
 1.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 2.0,
 4.0,
 5.0,
 4.0,
 4.0,
 4.0]

Now we will practice how we will convert this into a dataframe. First we'll create a master dataframe which will contain all review text data. 

### Compile Dataframes

In [127]:
#test_df = pd.DataFrame({'business_name': business_name_test,
#                        'source_url': source_url_test,
#                        'review_ratings': [rating for rating in review_ratings_test],
#                        'review_text': [text for text in review_text_test]})

#test_df

In [128]:
# append test df to master_df
master_df = master_df.append(pd.DataFrame({'business_name': business_name_test,
                                           'source_url': source_url_test,
                                           'review_ratings': [rating for rating in review_ratings_test],
                                           'review_text': [text for text in review_text_test]}))

In [136]:
# make sure to reset index after appending dataframe
master_df = master_df.reset_index(drop=True)
master_df = master_df.drop(['level_0', 'index'], axis=1)
master_df

,business_name,source_url,review_ratings,review_text
0,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,1.0,Iced mocha and hot mocha were flavorless. Tast...
1,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,4.0,Good coffee. Didn't try any of the pastries. C...
2,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,Cozy atmosphere. REALLY GOOD almond croissants...
3,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,What a nice contemporary coffee shop. 3 min wa...
4,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,"Great coffee, nice baristas, and a cute place ..."
5,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,"Cozy new coffee shop and roaster in Eastlake,m..."
6,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,Love this place. Came in here for the first ti...
7,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,"Best coffee in Eastlake, bar none. Every baris..."
8,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,4.0,Good coffee and great entrepreneur. Had their...
9,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,Wish I could give 10 stars. I've drank espress...


## Test Single Yelp Page with Many Reviews

We don't want to be scraping more than 100 reviews for each business. Therefore, under the 'blocks' parameter we will list 100. 

In [ ]:
# profile website
url = "https://app.datashake.com/api/v2/profiles/add"

# url of yelp page to scrape
querystring = {"url":"https://www.yelp.com/biz/retreat-seattle-2?osq=coffee+shops",
               "blocks": 100}

# provide api key
headers = {
    'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
    }

# request and print response
response = requests.request("POST", url, headers=headers, params=querystring)
print(response.text)

Now we'll post a request for these reviews and add them to our master dataframe. This will be a loop which will loop through all pages of our results and add the data to our master dataframe. 

In [153]:
for page in [1,2,3,4,5]:    
    url = "https://app.datashake.com/api/v2/profiles/reviews"
    # specifiy pages to scrape
    #querystring = {"job_id":"{}".format(job_id_test),
    querystring = {"job_id":"118669464",
                   "page": page}
    headers = {
        'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
        }
    response_reviews = requests.request("GET", url, headers=headers, params=querystring)
    
    json_response = json.loads(response_reviews.text)
    
    # grab source url
    source_url = json_response['source_url']

    # grab business name
    business_name = json.loads(json_response['meta_data'])['name']
    
    # grab review rating
    review_ratings = []
    for review in json_response['reviews']:
        review_ratings.append(review['rating_value'])
    
    # grab review text data
    review_texts = []
    for review in json_response['reviews']:
        review_texts.append(review['review_text'])
    
    # append test df to master_df
    master_df = master_df.append(pd.DataFrame({'business_name': business_name,
                                               'source_url': source_url,
                                               'review_ratings': [rating for rating in review_ratings],
                                               'review_text': [text for text in review_texts]})).reset_index(drop=True)

It appears that our loop worked as expected. Now we need to make sure our code works on sites with less than 100 reviews. 

In [159]:
master_df = master_df.reset_index(drop=True)
master_df[50:100]

,business_name,source_url,review_ratings,review_text
50,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,4.0,How should I put this... <br><br>Setting: <br>...
51,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,4.0,If you like espresso drinks or coffee drinks w...
52,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,4.0,Cool ambiance and friendly staff. Prices are p...
53,Retreat,https://www.yelp.com/biz/retreat-seattle-2?osq...,5.0,I got the sweet potato pesto bowl and avocado ...
54,Retreat,https://www.yelp.com/biz/retreat-seattle-2?osq...,5.0,Can't believe I haven't been to this place til...
55,Retreat,https://www.yelp.com/biz/retreat-seattle-2?osq...,1.0,I love the location &amp; vibe of this place b...
56,Retreat,https://www.yelp.com/biz/retreat-seattle-2?osq...,5.0,Came here with my sister during her visit. Ove...
57,Retreat,https://www.yelp.com/biz/retreat-seattle-2?osq...,4.0,Came here for brunch on a weekend and loved th...
58,Retreat,https://www.yelp.com/biz/retreat-seattle-2?osq...,4.0,What an amazing pit stop location for a quick ...
59,Retreat,https://www.yelp.com/biz/retreat-seattle-2?osq...,4.0,"Just as its name implies, this spot really is ..."


## Test Single Yelp Page with Less Than 100 Reviews

1. Make sure sites with less than 100 reviews you are not charged 110 tokens. 
2. Make sure there are no bugs in for loop when appending data to master_df. 

In [160]:
# profile website
url = "https://app.datashake.com/api/v2/profiles/add"

# url of yelp page to scrape
querystring_3 = {"url":"https://www.yelp.com/biz/caffe-fiore-west-seattle-seattle?osq=coffee+shops",
               "blocks": 100}

# provide api key
headers = {
    'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
    }

# request and print response
response_3 = requests.request("POST", url, headers=headers, params=querystring_3)
print(response.text)

{"success":true,"job_id":118669464,"status":200,"message":"Added this profile to the queue..."}


In [161]:
job_id_3 = json.loads(response_3.text)['job_id']

In [162]:
for page in [1,2,3,4,5]:    
    url = "https://app.datashake.com/api/v2/profiles/reviews"
    # specifiy pages to scrape
    querystring = {"job_id":"{}".format(job_id_3),
                   "page": page}
    headers = {
        'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
        }
    response_reviews = requests.request("GET", url, headers=headers, params=querystring)
    
    json_response = json.loads(response_reviews.text)
    
    # grab source url
    source_url = json_response['source_url']

    # grab business name
    business_name = json.loads(json_response['meta_data'])['name']
    
    # grab review rating
    review_ratings = []
    for review in json_response['reviews']:
        review_ratings.append(review['rating_value'])
    
    # grab review text data
    review_texts = []
    for review in json_response['reviews']:
        review_texts.append(review['review_text'])
    
    # append test df to master_df
    master_df = master_df.append(pd.DataFrame({'business_name': business_name,
                                               'source_url': source_url,
                                               'review_ratings': [rating for rating in review_ratings],
                                               'review_text': [text for text in review_texts]})).reset_index(drop=True)

In [163]:
master_df

,business_name,source_url,review_ratings,review_text
0,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,1.0,Iced mocha and hot mocha were flavorless. Tast...
1,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,4.0,Good coffee. Didn't try any of the pastries. C...
2,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,Cozy atmosphere. REALLY GOOD almond croissants...
3,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,What a nice contemporary coffee shop. 3 min wa...
4,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,"Great coffee, nice baristas, and a cute place ..."
...,...,...,...,...
211,Caffe Fiore West Seattle,https://www.yelp.com/biz/caffe-fiore-west-seat...,1.0,I must start off by saying the espresso is ver...
212,Caffe Fiore West Seattle,https://www.yelp.com/biz/caffe-fiore-west-seat...,5.0,Here now! I'd been going to Freshy's for a wh...
213,Caffe Fiore West Seattle,https://www.yelp.com/biz/caffe-fiore-west-seat...,5.0,"Coffee is wonderful, not too acidic but has en..."
214,Caffe Fiore West Seattle,https://www.yelp.com/biz/caffe-fiore-west-seat...,4.0,Just found this place today. It is my favorit...


This worked as expected. Move on to scraping ALL coffee shops in Seattle. 

# Scrape All Seattle Coffee Shops

In [244]:
# create list of URL's we still need to scrape
scrape_df_seattle_coffee = scrape_df[(scrape_df['Category'] == 'Coffee Shop') & (scrape_df['Area'] == 'Seattle / Belleve, WA')]
seattle_coffee_url_list = list(scrape_df_seattle_coffee['URL'])
seattle_coffee_url_list

['https://www.yelp.com/biz/amore-coffee-bellevue?osq=coffee+shops',
 'https://www.yelp.com/biz/ampersand-cafe-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/analog-coffee-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/anchored-ship-coffee-bar-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/anchorhead-coffee-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/armistice-coffee-roaster-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/arosa-cafe-seattle-6?osq=coffee+shops',
 'https://www.yelp.com/biz/ballard-coffee-works-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/bark-espresso-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/bedlam-coffee-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/blackbird-seattle-bellevue-3?osq=coffee+shops',
 'https://www.yelp.com/biz/broadcast-coffee-roasters-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/caf%C3%A9-besalu-seattle-2?osq=coffee+shops',
 'https://www.yelp.com/biz/cafe-anclair-seattle?osq=coffee+

In [245]:
# we've already scraped these URL's
already_scraped = ['https://www.yelp.com/biz/armistice-coffee-roaster-seattle?osq=coffee+shops',
                   'https://www.yelp.com/biz/retreat-seattle-2?osq=coffee+shops',
                   'https://www.yelp.com/biz/caffe-fiore-west-seattle-seattle?osq=coffee+shops']

In [246]:
# filter list for urls which were already scraped
seattle_coffee_url_list = list(filter(lambda x: x not in already_scraped, seattle_coffee_url_list))

## Create Loop to Request Seattle Coffee Review Data Scrape

In [207]:
# initialize list of job ids
seattle_coffee_job_ids = []

In [213]:
seattle_coffee_url_list

['https://www.yelp.com/biz/amore-coffee-bellevue?osq=coffee+shops',
 'https://www.yelp.com/biz/ampersand-cafe-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/analog-coffee-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/anchored-ship-coffee-bar-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/anchorhead-coffee-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/arosa-cafe-seattle-6?osq=coffee+shops',
 'https://www.yelp.com/biz/ballard-coffee-works-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/bark-espresso-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/bedlam-coffee-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/blackbird-seattle-bellevue-3?osq=coffee+shops',
 'https://www.yelp.com/biz/broadcast-coffee-roasters-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/caf%C3%A9-besalu-seattle-2?osq=coffee+shops',
 'https://www.yelp.com/biz/cafe-anclair-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/caff%C3%A8-senso-unico-seattle-4?osq=coffee+

In [214]:
# iterate through list of urls with for loop
for url in seattle_coffee_url_list:    
    request_url = "https://app.datashake.com/api/v2/profiles/add"

    # url of yelp page to scrape
    querystring = {"url": url,
                   "blocks": 100}

    # provide api key
    headers = {
        'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
        }

    # request and print response
    response = requests.request("POST", request_url, headers=headers, params=querystring)
    print(response.text)
    
    # obtain job_id and append to list
    job_id = json.loads(response.text)['job_id']
    seattle_coffee_job_ids.append(job_id)

{"success":true,"job_id":118678124,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678125,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678126,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678127,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678128,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678129,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678130,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678131,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678133,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678134,"status":200,"message":"Added this profile to the queue..."}
{"success":false,"status":402,"message":

KeyError: 'job_id'

### Clean Up Mess

It appears that by running multiple jobs we made the API believe we were exceeding our number of available credits. We will try to add a timer to slow down the requests so that that does not happen again. First we need to extract this data and add it to our seattle coffee reviews dataframe. 

In [215]:
seattle_coffee_reviews_df = master_df
seattle_coffee_reviews_df.head()

,business_name,source_url,review_ratings,review_text
0,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,1.0,Iced mocha and hot mocha were flavorless. Tast...
1,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,4.0,Good coffee. Didn't try any of the pastries. C...
2,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,Cozy atmosphere. REALLY GOOD almond croissants...
3,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,What a nice contemporary coffee shop. 3 min wa...
4,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,"Great coffee, nice baristas, and a cute place ..."


In [218]:
# iterate through job ids
for id in seattle_coffee_job_ids:
    # iterate through pages of results
    for page in [1,2,3,4,5]:    
        # specify url
        url = "https://app.datashake.com/api/v2/profiles/reviews"
        
        # specifiy pages to scrape
        querystring = {"job_id":"{}".format(id),
                       "page": page}
        headers = {
            'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
            }
        response_reviews = requests.request("GET", url, headers=headers, params=querystring)

        json_response = json.loads(response_reviews.text)

        # grab source url
        source_url = json_response['source_url']

        # grab business name
        business_name = json.loads(json_response['meta_data'])['name']

        # grab review rating
        review_ratings = []
        for review in json_response['reviews']:
            review_ratings.append(review['rating_value'])

        # grab review text data
        review_texts = []
        for review in json_response['reviews']:
            review_texts.append(review['review_text'])

        # append test df to master_df
        seattle_coffee_reviews_df = seattle_coffee_reviews_df.append(pd.DataFrame({'business_name': business_name,
                                                                                   'source_url': source_url,
                                                                                   'review_ratings': [rating for rating in review_ratings],
                                                                                   'review_text': [text for text in review_texts]})).reset_index(drop=True)

In [221]:
seattle_coffee_reviews_df

,business_name,source_url,review_ratings,review_text
0,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,1.0,Iced mocha and hot mocha were flavorless. Tast...
1,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,4.0,Good coffee. Didn't try any of the pastries. C...
2,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,Cozy atmosphere. REALLY GOOD almond croissants...
3,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,What a nice contemporary coffee shop. 3 min wa...
4,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,"Great coffee, nice baristas, and a cute place ..."
...,...,...,...,...
1179,Blackbird Seattle,https://www.yelp.com/biz/blackbird-seattle-bel...,5.0,Durian Mille Crepe ($9)<br> Seasame Mille Crep...
1180,Blackbird Seattle,https://www.yelp.com/biz/blackbird-seattle-bel...,5.0,This is a hidden gem for now. We tried the ber...
1181,Blackbird Seattle,https://www.yelp.com/biz/blackbird-seattle-bel...,5.0,So excited to have stumbled upon this hidden g...
1182,Blackbird Seattle,https://www.yelp.com/biz/blackbird-seattle-bel...,4.0,"This is a delightful quaint shop with coffee, ..."


In [253]:
# obtain list of unique values for url's which have already been scraped
already_scraped = list(seattle_coffee_reviews_df['source_url'].unique())

In [255]:
already_scraped

['https://www.yelp.com/biz/armistice-coffee-roaster-seattle?osq=coffee shops',
 'https://www.yelp.com/biz/retreat-seattle-2?osq=coffee shops',
 'https://www.yelp.com/biz/caffe-fiore-west-seattle-seattle?osq=coffee shops',
 'https://www.yelp.com/biz/amore-coffee-bellevue?osq=coffee shops',
 'https://www.yelp.com/biz/ampersand-cafe-seattle?osq=coffee shops',
 'https://www.yelp.com/biz/analog-coffee-seattle?osq=coffee shops',
 'https://www.yelp.com/biz/anchored-ship-coffee-bar-seattle?osq=coffee shops',
 'https://www.yelp.com/biz/anchorhead-coffee-seattle?osq=coffee shops',
 'https://www.yelp.com/biz/arosa-cafe-seattle-6?osq=coffee shops',
 'https://www.yelp.com/biz/ballard-coffee-works-seattle?osq=coffee shops',
 'https://www.yelp.com/biz/bark-espresso-seattle?osq=coffee shops',
 'https://www.yelp.com/biz/bedlam-coffee-seattle?osq=coffee shops',
 'https://www.yelp.com/biz/blackbird-seattle-bellevue-3?osq=coffee shops']

In [263]:
for url in already_scraped:
    url = str(url).replace(" ","+")

In [275]:
for i in range(len(already_scraped)):
    already_scraped[i] = already_scraped[i].replace(" ","+")
already_scraped

['https://www.yelp.com/biz/armistice-coffee-roaster-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/retreat-seattle-2?osq=coffee+shops',
 'https://www.yelp.com/biz/caffe-fiore-west-seattle-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/amore-coffee-bellevue?osq=coffee+shops',
 'https://www.yelp.com/biz/ampersand-cafe-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/analog-coffee-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/anchored-ship-coffee-bar-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/anchorhead-coffee-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/arosa-cafe-seattle-6?osq=coffee+shops',
 'https://www.yelp.com/biz/ballard-coffee-works-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/bark-espresso-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/bedlam-coffee-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/blackbird-seattle-bellevue-3?osq=coffee+shops']

In [276]:
# filter list for urls which were already scraped
seattle_coffee_url_list = list(filter(lambda x: x not in already_scraped, seattle_coffee_url_list))

In [281]:
len(seattle_coffee_url_list)

68

## Create Loop to Request Seattle Coffee Review Data Scrape - Take 2

In [280]:
# wait time
time.sleep(10)
print("10 seconds to execute?")

10 seconds to execute?


In [282]:
# iterate through list of urls with for loop
for url in seattle_coffee_url_list:    
    request_url = "https://app.datashake.com/api/v2/profiles/add"

    # url of yelp page to scrape
    querystring = {"url": url,
                   "blocks": 100}

    # provide api key
    headers = {
        'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
        }

    # request and print response
    response = requests.request("POST", request_url, headers=headers, params=querystring)
    print(response.text)
    
    # obtain job_id and append to list
    job_id = json.loads(response.text)['job_id']
    seattle_coffee_job_ids.append(job_id)
    
    # add 60 second delay before making next request
    time.sleep(10)

{"success":true,"job_id":118678703,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678708,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678712,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678719,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678722,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678727,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678730,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678734,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678736,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678744,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118678746,"stat

KeyError: 'job_id'

### Clean Up Mess Again

In [284]:
# iterate through job ids
for id in seattle_coffee_job_ids:
    # iterate through pages of results
    for page in [1,2,3,4,5]:    
        # specify url
        url = "https://app.datashake.com/api/v2/profiles/reviews"
        
        # specifiy pages to scrape
        querystring = {"job_id":"{}".format(id),
                       "page": page}
        headers = {
            'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
            }
        response_reviews = requests.request("GET", url, headers=headers, params=querystring)

        json_response = json.loads(response_reviews.text)

        # grab source url
        source_url = json_response['source_url']

        # grab business name
        business_name = json.loads(json_response['meta_data'])['name']

        # grab review rating
        review_ratings = []
        for review in json_response['reviews']:
            review_ratings.append(review['rating_value'])

        # grab review text data
        review_texts = []
        for review in json_response['reviews']:
            review_texts.append(review['review_text'])

        # append test df to master_df
        seattle_coffee_reviews_df = seattle_coffee_reviews_df.append(pd.DataFrame({'business_name': business_name,
                                                                                   'source_url': source_url,
                                                                                   'review_ratings': [rating for rating in review_ratings],
                                                                                   'review_text': [text for text in review_texts]})).reset_index(drop=True)

In [286]:
# obtain list of unique values for url's which have already been scraped
already_scraped = list(seattle_coffee_reviews_df['source_url'].unique())

In [287]:
for i in range(len(already_scraped)):
    already_scraped[i] = already_scraped[i].replace(" ","+")
already_scraped

['https://www.yelp.com/biz/armistice-coffee-roaster-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/retreat-seattle-2?osq=coffee+shops',
 'https://www.yelp.com/biz/caffe-fiore-west-seattle-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/amore-coffee-bellevue?osq=coffee+shops',
 'https://www.yelp.com/biz/ampersand-cafe-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/analog-coffee-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/anchored-ship-coffee-bar-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/anchorhead-coffee-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/arosa-cafe-seattle-6?osq=coffee+shops',
 'https://www.yelp.com/biz/ballard-coffee-works-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/bark-espresso-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/bedlam-coffee-seattle?osq=coffee+shops',
 'https://www.yelp.com/biz/blackbird-seattle-bellevue-3?osq=coffee+shops',
 'https://www.yelp.com/biz/broadcast-coffee-roasters-seattle?osq=coff

In [288]:
# filter list for urls which were already scraped
seattle_coffee_url_list = list(filter(lambda x: x not in already_scraped, seattle_coffee_url_list))

In [289]:
len(seattle_coffee_url_list)

43

## Create Loop to Request Seattle Coffee Review Data Scrape - Take 3

In [290]:
# iterate through list of urls with for loop
for url in seattle_coffee_url_list:    
    request_url = "https://app.datashake.com/api/v2/profiles/add"

    # url of yelp page to scrape
    querystring = {"url": url,
                   "blocks": 100}

    # provide api key
    headers = {
        'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
        }

    # request and print response
    response = requests.request("POST", request_url, headers=headers, params=querystring)
    print(response.text)
    
    # obtain job_id and append to list
    job_id = json.loads(response.text)['job_id']
    seattle_coffee_job_ids.append(job_id)
    
    # add 50 second delay before making next request
    time.sleep(50)

{"success":true,"job_id":118678989,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679012,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679035,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679055,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679071,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679091,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679110,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679129,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679149,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679171,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679191,"stat

## Add All Coffee Reviews to Dataframe

In [291]:
# iterate through job ids
for id in seattle_coffee_job_ids:
    # iterate through pages of results
    for page in [1,2,3,4,5]:    
        # specify url
        url = "https://app.datashake.com/api/v2/profiles/reviews"
        
        # specifiy pages to scrape
        querystring = {"job_id":"{}".format(id),
                       "page": page}
        headers = {
            'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
            }
        response_reviews = requests.request("GET", url, headers=headers, params=querystring)

        json_response = json.loads(response_reviews.text)

        # grab source url
        source_url = json_response['source_url']

        # grab business name
        business_name = json.loads(json_response['meta_data'])['name']

        # grab review rating
        review_ratings = []
        for review in json_response['reviews']:
            review_ratings.append(review['rating_value'])

        # grab review text data
        review_texts = []
        for review in json_response['reviews']:
            review_texts.append(review['review_text'])

        # append review data to dataframe
        seattle_coffee_reviews_df = seattle_coffee_reviews_df.append(pd.DataFrame({'business_name': business_name,
                                                                                   'source_url': source_url,
                                                                                   'review_ratings': [rating for rating in review_ratings],
                                                                                   'review_text': [text for text in review_texts]})).reset_index(drop=True)

We had some shenanigans earlier so let's make sure to drop any duplicate review data that might have popped up into our dataframe.

In [295]:
# drop duplicate rows
seattle_coffee_reviews_df = seattle_coffee_reviews_df.drop_duplicates(subset='review_text').reset_index(drop=True)

In [296]:
seattle_coffee_reviews_df

,business_name,source_url,review_ratings,review_text
0,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,1.0,Iced mocha and hot mocha were flavorless. Tast...
1,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,4.0,Good coffee. Didn't try any of the pastries. C...
2,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,Cozy atmosphere. REALLY GOOD almond croissants...
3,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,What a nice contemporary coffee shop. 3 min wa...
4,Armistice Coffee Roaster,https://www.yelp.com/biz/armistice-coffee-roas...,5.0,"Great coffee, nice baristas, and a cute place ..."
...,...,...,...,...
7473,Zoka Coffee Roaster & Tea,https://www.yelp.com/biz/zoka-coffee-roaster-a...,5.0,"First off, the coffee is to die for. Particula..."
7474,Zoka Coffee Roaster & Tea,https://www.yelp.com/biz/zoka-coffee-roaster-a...,5.0,This review is an update to an earlier post I ...
7475,Zoka Coffee Roaster & Tea,https://www.yelp.com/biz/zoka-coffee-roaster-a...,4.0,Gluten-free chocolate decadence and non-fat gi...
7476,Zoka Coffee Roaster & Tea,https://www.yelp.com/biz/zoka-coffee-roaster-a...,4.0,"COFFEE: Not the best coffee, the the atmospher..."


In [297]:
# finally we'll export this data to a csv
seattle_coffee_reviews_df.to_csv('seattle_coffee_reviews_df.csv', index=False)

# Scrape All Seattle Boutique Shops

In [298]:
# create boutiques dataframe we can add review data to 
seattle_boutique_reviews_df = pd.DataFrame(columns = ['business_name','source_url', 'review_ratings','review_text'])

In [301]:
# compile list of url's to scrape
scrape_df_seattle_boutique = scrape_df[(scrape_df['Category'] == 'Niche Shopping') & (scrape_df['Area'] == 'Seattle / Belleve, WA')]
seattle_boutique_url_list = list(scrape_df_seattle_boutique['URL'])
seattle_boutique_url_list

['https://www.yelp.com/biz/alexandras-designer-consignment-seattle?osq=vintage',
 'https://www.yelp.com/biz/andaluz-seattle?osq=gift+shop',
 'https://www.yelp.com/biz/ballard-consignment-store-seattle-2?osq=vintage',
 'https://www.yelp.com/biz/beats-and-bohos-seattle-2?osq=vintage',
 'https://www.yelp.com/biz/bon-voyage-vintage-seattle?osq=vintage',
 'https://www.yelp.com/biz/casita-international-seattle?osq=gift+shop',
 'https://www.yelp.com/biz/clementines-seattle?osq=boutique',
 'https://www.yelp.com/biz/couth-buzzard-books-espresso-buono-seattle?osq=book+store',
 'https://www.yelp.com/biz/destee-nation-seattle?osq=vintage',
 'https://www.yelp.com/biz/diva-dollz-seattle?osq=vintage',
 'https://www.yelp.com/biz/drizzle-and-shine-seattle?osq=boutique',
 'https://www.yelp.com/biz/fleurt-collective-seattle?osq=gift+shop',
 'https://www.yelp.com/biz/fremont-vintage-mall-seattle?osq=vintage',
 'https://www.yelp.com/biz/glasswing-seattle?osq=vintage',
 'https://www.yelp.com/biz/gold-dogs-s

## Create Loop to Request Seattle Boutique Review Data Scrape

In [302]:
# initialize list of job ids
seattle_boutique_job_ids = []

In [303]:
# iterate through list of urls with for loop
for url in seattle_boutique_url_list:    
    request_url = "https://app.datashake.com/api/v2/profiles/add"

    # url of yelp page to scrape
    querystring = {"url": url,
                   "blocks": 100}

    # provide api key
    headers = {
        'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
        }

    # request and print response
    response = requests.request("POST", request_url, headers=headers, params=querystring)
    print(response.text)
    
    # obtain job_id and append to list
    job_id = json.loads(response.text)['job_id']
    seattle_boutique_job_ids.append(job_id)
    
    # add 50 second delay before making next request
    time.sleep(50)

{"success":true,"job_id":118679465,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679471,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679473,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679474,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679481,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679483,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679488,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679489,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679491,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679493,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":118679495,"stat

## Add All Boutique Reviews to Dataframe

In [304]:
# iterate through job ids
for id in seattle_boutique_job_ids:
    # iterate through pages of results
    for page in [1,2,3,4,5]:    
        # specify url
        url = "https://app.datashake.com/api/v2/profiles/reviews"
        
        # specifiy pages to scrape
        querystring = {"job_id":"{}".format(id),
                       "page": page}
        headers = {
            'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
            }
        
        # request reviews
        response_reviews = requests.request("GET", url, headers=headers, params=querystring)
        
        # convert response to json format
        json_response = json.loads(response_reviews.text)

        # grab source url
        source_url = json_response['source_url']

        # grab business name
        business_name = json.loads(json_response['meta_data'])['name']

        # grab review rating
        review_ratings = []
        for review in json_response['reviews']:
            review_ratings.append(review['rating_value'])

        # grab review text data
        review_texts = []
        for review in json_response['reviews']:
            review_texts.append(review['review_text'])

        # append test df to master_df
        seattle_boutique_reviews_df = seattle_boutique_reviews_df.append(pd.DataFrame({'business_name': business_name,
                                                                                       'source_url': source_url,
                                                                                       'review_ratings': [rating for rating in review_ratings],
                                                                                       'review_text': [text for text in review_texts]})).reset_index(drop=True)

In [305]:
# finally we'll export this data to a csv
seattle_boutique_reviews_df.to_csv('seattle_boutique_reviews_df.csv', index=False)

# Scrape All Seattle Adult Entertainment Businesses

In [309]:
# create adult entertainment dataframe we can add review data to 
seattle_adult_reviews_df = pd.DataFrame(columns = ['business_name','source_url', 'review_ratings','review_text'])

In [310]:
# compile list of url's to scrape
scrape_df_seattle_adult = scrape_df[(scrape_df['Category'] == 'Adult Entertainment') & 
                                    (scrape_df['Area'] == 'Seattle / Belleve, WA')]
seattle_adult_url_list = list(scrape_df_seattle_adult['URL'])
seattle_adult_url_list

['https://www.yelp.com/biz/white-horse-tavern-seattle?osq=book+store',
 'https://www.yelp.com/biz/footprint-wine-tap-seattle?osq=wine+bars',
 'https://www.yelp.com/biz/bottlehouse-seattle?osq=wine+bars',
 'https://www.yelp.com/biz/left-bank-seattle-2?osq=wine+bars',
 'https://www.yelp.com/biz/house-of-pagne-seattle?osq=wine+bars',
 'https://www.yelp.com/biz/marseille-seattle-2?osq=wine+bars',
 'https://www.yelp.com/biz/the-tasting-room-seattle?osq=wine+bars',
 'https://www.yelp.com/biz/the-barrel-thief-seattle-2?osq=wine+bars',
 'https://www.yelp.com/biz/claret-wine-bar-seattle?osq=wine+bars',
 'https://www.yelp.com/biz/belmont-seattle?osq=wine+bars',
 'https://www.yelp.com/biz/the-sitting-room-seattle?osq=wine+bars',
 'https://www.yelp.com/biz/aluel-cellars-seattle-2?osq=wine+bars',
 'https://www.yelp.com/biz/locol-barley-and-vine-seattle-3?osq=wine+bars',
 'https://www.yelp.com/biz/browne-family-vineyards-seattle?osq=wine+bars',
 'https://www.yelp.com/biz/northwest-tastings-wine-bar-

## Create Loop to Request Seattle Adult Entertainment Review Data

In [311]:
# initialize list of job ids
seattle_adult_job_ids = []

In [312]:
# iterate through list of urls with for loop
for url in seattle_adult_url_list:    
    request_url = "https://app.datashake.com/api/v2/profiles/add"

    # url of yelp page to scrape
    querystring = {"url": url,
                   "blocks": 100}

    # provide api key
    headers = {
        'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
        }

    # request and print response
    response = requests.request("POST", request_url, headers=headers, params=querystring)
    print(response.text)
    
    # obtain job_id and append to list
    job_id = json.loads(response.text)['job_id']
    seattle_adult_job_ids.append(job_id)
    
    # add 50 second delay before making next request
    time.sleep(50)

{"success":true,"job_id":119755753,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119755754,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119755756,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119755757,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119755765,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119755774,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119755847,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119755852,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119755857,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119755869,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119755884,"stat

{"success":true,"job_id":119756808,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119756814,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119756815,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119756816,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119756819,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119756826,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119756829,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119756833,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119756835,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119756837,"status":200,"message":"Added this profile to the queue..."}
{"success":true,"job_id":119756842,"stat

## Add All Adult Entertainment Reviews to Dataframe

In [313]:
# iterate through job ids
for id in seattle_adult_job_ids:
    # iterate through pages of results
    for page in [1,2,3,4,5]:    
        # specify url
        url = "https://app.datashake.com/api/v2/profiles/reviews"
        
        # specifiy pages to scrape
        querystring = {"job_id":"{}".format(id),
                       "page": page}
        headers = {
            'spiderman-token': "a977b8454ca8cc4324dda1aaf0742d09ff8e72ec",
            }
        
        # request reviews
        response_reviews = requests.request("GET", url, headers=headers, params=querystring)
        
        # convert response to json format
        json_response = json.loads(response_reviews.text)

        # grab source url
        source_url = json_response['source_url']

        # grab business name
        business_name = json.loads(json_response['meta_data'])['name']

        # grab review rating
        review_ratings = []
        for review in json_response['reviews']:
            review_ratings.append(review['rating_value'])

        # grab review text data
        review_texts = []
        for review in json_response['reviews']:
            review_texts.append(review['review_text'])

        # append test df to master_df
        seattle_adult_reviews_df = seattle_adult_reviews_df.append(pd.DataFrame({'business_name': business_name,
                                                                                 'source_url': source_url,
                                                                                 'review_ratings': [rating for rating in review_ratings],
                                                                                 'review_text': [text for text in review_texts]})).reset_index(drop=True)

In [314]:
# finally we'll export this data to a csv
seattle_adult_reviews_df.to_csv('seattle_adult_reviews_df.csv', index=False)